# Calibration and pose extraction testing

1. record video with calibration marker (checkerboard grid)
2. extract checkerboard and calibrate cameras
3. record video with stereo video of movement
4. extract pose from each camera

In [ ]:
import sys
import os

import numpy as np
import pandas as pd
import pickle

import cv2

# Video recording

In [ ]:
def synchronized_recording(output_video_file, size=(640, 480), fps=30):
    """ Acquire data from two webcams as synchronously as possible
    
        Creates a video file with the two videos vertically concatenated.
    """
    import cv2
    import numpy as np
    import time
    
    win, hin = size
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_file, fourcc, fps, (int(win), int(hin * 2)))

    # open camera
    cam1 = cv2.VideoCapture(0)
    cam2 = cv2.VideoCapture(1)

    cam1.set(cv2.CAP_PROP_AUTOFOCUS, 0)
    cam1.set(cv2.CAP_PROP_FPS, fps)
    cam1.set(cv2.CAP_PROP_FRAME_WIDTH, int(win))
    cam1.set(cv2.CAP_PROP_FRAME_HEIGHT, int(hin))

    cam2.set(cv2.CAP_PROP_AUTOFOCUS, 0)    
    cam2.set(cv2.CAP_PROP_FPS, fps)
    cam2.set(cv2.CAP_PROP_FRAME_WIDTH, int(win))
    cam2.set(cv2.CAP_PROP_FRAME_HEIGHT, int(hin))
    
    timestamps = []

    while True:

        ret1 = cam1.grab()
        ret2 = cam2.grab()

        if not ret1 or not ret2:
            print("Something is wrong with camera")
            break

        ret1, img1 = cam1.retrieve()
        ret2, img2 = cam2.retrieve()
        
        if not ret1 or img1 is None:
            print('Missed img1')
            continue
        if not ret2 or img2 is None:
            print('Missed img2')
            continue
            
        timestamps.append(time.time())
        combined = np.concatenate((img1, img2), axis=0)

        out.write(combined)
        
        cv2.imshow('combined', combined)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break                
            
    cv2.destroyAllWindows()
    
    return timestamps

In [ ]:
def parse_stereo_video(filename, callback):
    """ Process a stereo video
    
        Expects the format to match that from synchronized_recording and
        pass the top and bottom parts to the callback to analyze. The
        callback is expected to handle its own results.
    """
    import cv2
    import numpy as np
    
    from tqdm import tqdm
    
    vid = cv2.VideoCapture(filename)

    win = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
    hin = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = vid.get(cv2.CAP_PROP_FPS)
    frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in tqdm(range(frames)):
        ret, im = vid.read()
        top, bottom = np.split(im, 2, axis=0)
        
        ret = callback(top, bottom, i)
        if ret is False:
            break

# Calibrate stereo cameras

Obtain video of a checkerboard from two cameras and compute the intrinsic and extrinsic camera properties. When running the calibration routine, ensure that the checkboard vertices are well localized or the calibration will be poor. If needed, print a much larger grid.

In [ ]:
timestamps = synchronized_recording('checkerboard_calibration.mp4')

In [ ]:
# now detect the checkerboard in all frames

import cv2
import numpy as np

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 3, 0.001)

# NOTE: change the spacing below for a different grid size
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*8,3), np.float32)
objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2) * 2.3

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
im1points = [] # 2d points in image plane.
im2points = [] # 2d points in image plane.

def get_checkerboard(im):
    # Find the chess board corners
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (8, 6), None)
    if ret:
        return cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
    else:
        return None

def frame_checkboards(left, right, i, plot=True):
    c1 = get_checkerboard(left)
    c2 = get_checkerboard(right)
    
    if i % 10 != 0: return True
    
    if c1 is not None and c2 is not None:
        objpoints.append(objp)
        im1points.append(c1)
        im2points.append(c2)
        
        if plot:
            o1 = cv2.drawChessboardCorners(left, (8,6), c1, True)
            o2 = cv2.drawChessboardCorners(right, (8,6), c2, True)
                
            combined = np.concatenate((o1, o2), axis=0)
            cv2.imshow('combined', combined)
            if cv2.waitKey(1) == ord('q'): return False
            
    return len(objpoints) < 100

parse_stereo_video('checkerboard_calibration.mp4', frame_checkboards)

cv2.destroyAllWindows()

len(objpoints)

In [ ]:
# independently calibrate each camera based on the 

def calibrate(objpoints, imgpoints):
    h, w, d = (640, 480, 3)
    ret, mtx, dist, _, _ = cv2.calibrateCamera(objpoints, imgpoints, (w, h), None, None)
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
    return {'mtx': mtx, 'dist': dist, 'newcameramtx': newcameramtx, 'roi': roi}

if True:
    print('Calibrating')
    calibration1 = calibrate(objpoints[::1], im1points[::1])
    calibration2 = calibrate(objpoints[::1], im2points[::1])

    print(calibration1)
    print(calibration2)

In [ ]:
# now perform a stereo calibration which refines the intrinsics (optional)
# and computes the extrinsic calibration

h, w, d = (640, 480, 3)
stereocalib_criteria = (cv2.TERM_CRITERIA_EPS, 100, 1e-5)

# note there are many components of the camera view that can be calibrated. In this case using
# the initial guess from the prior step and forcing the same focal length as I am using identical
# cameras

#stereocalib_flags = 0
#stereocalib_flags = cv2.CALIB_FIX_INTRINSIC
#stereocalib_flags = cv2.CALIB_SAME_FOCAL_LENGTH
#stereocalib_flags = cv2.CALIB_FIX_ASPECT_RATIO | cv2.CALIB_SAME_FOCAL_LENGTH
#stereocalib_flags = cv2.CALIB_SAME_FOCAL_LENGTH | cv2.CALIB_FIX_INTRINSIC | cv2.CALIB_ZERO_TANGENT_DIST #cv2.CALIB_ZERO_TANGENT_DIST | cv2.CALIB_SAME_FOCAL_LENGTH | cv2.CALIB_RATIONAL_MODEL | cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5
#stereocalib_flags = cv2.CALIB_SAME_FOCAL_LENGTH | cv2.CALIB_ZERO_TANGENT_DIST | cv2.CALIB_SAME_FOCAL_LENGTH | cv2.CALIB_RATIONAL_MODEL | cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5
#stereocalib_flags = cv2.CALIB_SAME_FOCAL_LENGTH | cv2.CALIB_FIX_K1 | cv2.CALIB_FIX_K2 | cv2.CALIB_FIX_K3 | cv2.CALIB_ZERO_TANGENT_DIST
stereocalib_flags = cv2.CALIB_USE_INTRINSIC_GUESS | cv2.CALIB_SAME_FOCAL_LENGTH # actually this worked  great!


res = cv2.stereoCalibrate(objpoints, im1points, im2points,
                          calibration1['mtx'].copy(), calibration1['dist'].copy() * 0,
                          calibration2['mtx'].copy(), calibration2['dist'].copy() * 0,
                          (w, h),                          
                          criteria = stereocalib_criteria,
                          flags = stereocalib_flags)
stereocalib_retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F = res

data = {'cameraMatrix1': cameraMatrix1, 'distCoeffs1': distCoeffs1,
        'cameraMatrix2': cameraMatrix2, 'distCoeffs2': distCoeffs2,
        'R': R, 'T': T, 'E': E, 'F': F}

print(stereocalib_retval)
[print(k, '\n', data[k]) for k in data.keys()]

In [ ]:

rectify_scale = 1.0 # 0=full crop, 1=no crop

R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(data["cameraMatrix1"], data["distCoeffs1"],
                                                  data["cameraMatrix2"], data["distCoeffs2"],
                                                  (w, h), data["R"], data["T"], alpha=rectify_scale)
#, flags=0*cv2.CALIB_ZERO_DISPARITY,
#                                                  alpha = rectify_scale)

data['R1'] = R1
data['R2'] = R2
data['P1'] = P1
data['P2'] = P2
data['Q'] = Q
data['roi1'] = roi1
data['roi2'] = roi2

print(roi1)
print(roi2)
print(P1)
print(P2)

left_maps = cv2.initUndistortRectifyMap(data["cameraMatrix1"], data["distCoeffs1"], R1, P1, (w, h), cv2.CV_16SC2)
right_maps = cv2.initUndistortRectifyMap(data["cameraMatrix2"], data["distCoeffs2"], R2, P2, (w, h), cv2.CV_16SC2)

def play_rectified(left, right, i):
    left_img_remap = cv2.remap(left, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
    right_img_remap = cv2.remap(right, right_maps[0], right_maps[1], cv2.INTER_LANCZOS4) #cv2.INTER_LANCZOS4)
    
    #combined = left_img_remap
    combined = np.concatenate((left_img_remap, right_img_remap), axis=0)
    #combined = np.concatenate((left, right), axis=0)

    cv2.imshow('combined', combined)

    key = cv2.waitKey(1)
    return key != ord('q')       

parse_stereo_video('checkerboard_calibration.mp4', play_rectified)
cv2.destroyAllWindows()

In [ ]:
# same the calibration data, which will be used in the fusion step

with open('calibration.pkl', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Record movement video for analysis

In [ ]:
# Note: You should be acquiring inertial data at the same time

timestamps = synchronized_recording('movement.mp4')
np.save('movement_timestamps', timestamps) # make backup in case of crash

# Process video with OpenPose

In [ ]:
# NOTE: you must compile openpose with the python bindings and make
# sure that jupyter can find it for the following code

home = os.path.expanduser("~")
openpose_python_path = os.path.join(home, 'projects/pose/openpose/build/python')

sys.path.append(openpose_python_path)

from openpose import pyopenpose as op

In [ ]:

def get_openpose_parser(results_path):
    params = {'model_folder': os.path.join(openpose_python_path, '../../models'),
              'number_people_max': 1, 'write_json': results_path}

    #params["body"] = 1

    #params["face"] = False
    #params["face_detector"] = 2

    #params["hand"] = False
    #params["hand_detector"] = 2

    opWrapper = op.WrapperPython()
    opWrapper.configure(params)
    opWrapper.start()
    
    return opWrapper

faceRectangles = [
    op.Rectangle(330.119385, 277.532715, 48.717274, 48.717274),
    op.Rectangle(24.036991, 267.918793, 65.175171, 65.175171),
    op.Rectangle(151.803436, 32.477852, 108.295761, 108.295761),
]

handRectangles = [
    # Left/Right hands person 0
    [
    op.Rectangle(320.035889, 377.675049, 69.300949, 69.300949),
    op.Rectangle(0., 0., 0., 0.),
    ],
    # Left/Right hands person 1
    [
    op.Rectangle(80.155792, 407.673492, 80.812706, 80.812706),
    op.Rectangle(46.449715, 404.559753, 98.898178, 98.898178),
    ],
    # Left/Right hands person 2
    [
    op.Rectangle(185.692673, 303.112244, 157.587555, 157.587555),
    op.Rectangle(88.984360, 268.866547, 117.818230, 117.818230),
    ]
]

OP_NOSE = 0
OP_NECK = 1
OP_RSHOULDER = 2
OP_RELBOW = 3
OP_RWRIST = 4
OP_LSHOULDER = 5
OP_LELBOW = 6
OP_LWRIST = 7
OP_MIDHIP = 8
OP_RHIP = 9
OP_RKNEE = 10
OP_RANKLE = 11
OP_LHIP = 12
OP_LKNEE = 13
OP_LANKLE = 14
OP_REYE = 15
OP_LEYE = 16
OP_LBIGTOE = 19
OP_LSMALLTOE = 20
OP_LHEEL = 21
OP_RBIGTOE = 22
OP_RSMALLTOE = 23
OP_RHEEL = 24

PARTS = ["Nose", "Neck", "RShoulder", "RElbow", "RWrist", "LShoulder", "LElbow", "LWrist", "MidHip", "RHip",
         "RKnee", "RAnkle", "LHip", "LKnee", "LAnkle", "REye", "LEye", "REar", "LEar", "LBigToe", "LSmallToe", 
         "LHeel", "RBigToe", "RSmallToe", "RHeel"]

In [ ]:
# process video

fn = f'movement.mp4'
timestamps = np.load(f'movement_timestamps.npy')

base = os.path.splitext(fn)[0]
out_fn = base + "_openpose.mp4"
pickle_file = base + ".pkl"
json_path = base + '_json'

opWrapper = get_openpose_parser(json_path)
import cv2
vid = cv2.VideoCapture(fn)

win = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
hin = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = vid.get(cv2.CAP_PROP_FPS)
total_frames = vid.get(cv2.CAP_PROP_FRAME_COUNT)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(out_fn, fourcc, fps, (int(win), int(hin)))

keypointsLeft = []
keypointsRight = []
allPoints = []

def process_frame(im):
    datum = op.Datum()
    datum.cvInputData = im
    datum.faceRectangles = faceRectangles
    datum.handRectangles = handRectangles
    opWrapper.emplaceAndPop([datum])
    
    return datum.cvOutputData, datum.poseKeypoints

def frame_pose_extraction(left, right, i, plot=True, save=True):
    ol1, kp1 = process_frame(left)
    ol2, kp2 = process_frame(right)

    keypointsLeft.append(kp1)
    keypointsRight.append(kp2)
    allPoints.append(np.concatenate((kp1.reshape(-1), kp2.reshape(-1))))
    
    combined = np.concatenate((ol1, ol2), axis=0)
    
    if plot:
        cv2.imshow('combined', combined)
        if cv2.waitKey(1) == ord('q'): return False
        
    if save:
        out.write(combined)
            
    return True


parse_stereo_video(fn, frame_pose_extraction)
out.release()
cv2.destroyAllWindows()

In [ ]:
# Save joint positions from each camera and confidences

columns = pd.MultiIndex.from_product([['camL', 'camR'], PARTS, ['x', 'y', 'c']], names=['side', 'part', 'component'])
timestamp_index = pd.DatetimeIndex([pd.Timestamp.fromtimestamp(t) for t in timestamps])
stereo_keypoints = pd.DataFrame(allPoints, columns=columns, index=timestamp_index)
stereo_keypoints.to_pickle(pickle_file)

# Triangulate pose into 3D coordinates

This is not really used subsequently, but demonstrates how the coordinates can be projected into 3D.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt

cal = pickle.load(open('calibration.pkl', 'rb'))

In [ ]:
parts = stereo_keypoints.columns.get_level_values(1).unique()
p3d = []

smoothed = stereo_keypoints.ewm(10).mean()

for p in parts:
    p1 = smoothed['camL'][p][['y', 'x']].values
    p2 = smoothed['camR'][p][['y', 'x']].values
        
    # compute 3d homogeneous
    p3 = cv2.triangulatePoints(cal['P1'], cal['P2'], p1.transpose(), p2.transpose()).transpose()
    p3 = cv2.convertPointsFromHomogeneous(p3)[:,0,:]
    p3d.append(p3)

p3d = np.concatenate(p3d, axis=1)

In [ ]:
columns = pd.MultiIndex.from_product([parts, ['x', 'y', 'z']], names=['part', 'component'])
p3d = pd.DataFrame(p3d, columns=columns, index=stereo_keypoints.index)

In [ ]:
ax = p3d[['RWrist', 'RKnee']].iloc[150:800].plot()